# Neural network to learn a function f

We use Optuna to do a type of Bayesian optimization of the hyperparameters of the model. We then train the model using these hyperparameters to approximate the function $f$ that we define in this notebook.

Use this first cell to convert the notebook to a python script.

In [22]:
!jupyter nbconvert script.ipynb --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}' --to script

[NbConvertApp] Converting notebook script.ipynb to script
[NbConvertApp] Writing 20039 bytes to script.py


In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import tensorboardX as tbx


In [14]:
%%script echo skipping
# Defining the function to approximate
def f(x1, x2, x3):
    return x1 + x2 + x3

# Generating the data samples
nsamples = 10**5

x1min = x2min = x3min = -10
x1max = x2max = x3max = 10

np.random.seed(0)
x1 = np.random.uniform(x1min, x1max, size=nsamples)
x2 = np.random.uniform(x2min, x2max, size=nsamples)
x3 = np.random.uniform(x3min, x3max, size=nsamples)
y = f(x1, x2, x3)

# Converting the data to tensors
x1 = torch.from_numpy(x1).float()
x2 = torch.from_numpy(x2).float()
x3 = torch.from_numpy(x3).float()
y = torch.from_numpy(y).float()

# Stacking the inputs into a single tensor
x = torch.stack([x1, x2, x3], dim=1)

# Splitting the data into train and test sets
train_size = int(0.8 * len(x))
test_size = len(x) - train_size
x_train, x_test = torch.split(x, [train_size, test_size])
y_train, y_test = torch.split(y, [train_size, test_size])


skipping


In [15]:
# Checking if GPU is available and setting the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generating the data
np.random.seed(0)
x = np.random.uniform(-10, 10, size=(10000, 3))
y = 2 * x[:, 0] - 3 * x[:, 1] + 5 * x[:, 2] + np.random.normal(0, 1, size=10000)

# Splitting the data into train and test sets
x_train = torch.tensor(x[:8000], dtype=torch.float32).to(device) # Added moving the tensor to the device
y_train = torch.tensor(y[:8000], dtype=torch.float32).to(device) # Added moving the tensor to the device
x_test = torch.tensor(x[8000:], dtype=torch.float32).to(device) # Added moving the tensor to the device
y_test = torch.tensor(y[8000:], dtype=torch.float32).to(device) # Added moving the tensor to the device


In [16]:

# Defining a class for the network
class Net(nn.Module):
    """A class for creating a network with a variable number of hidden layers and units.

    Attributes:
        n_layers (int): The number of hidden layers in the network.
        n_units (list): A list of integers representing the number of units in each hidden layer.
        hidden_activation (torch.nn.Module): The activation function for the hidden layers.
        output_activation (torch.nn.Module): The activation function for the output layer.
        layers (torch.nn.ModuleList): A list of linear layers in the network.
    """

    def __init__(self, n_layers, n_units, hidden_activation, output_activation):
        """Initializes the network with the given hyperparameters.

        Args:
            n_layers (int): The number of hidden layers in the network.
            n_units (list): A list of integers representing the number of units in each hidden layer.
            hidden_activation (torch.nn.Module): The activation function for the hidden layers.
            output_activation (torch.nn.Module): The activation function for the output layer.
        """
        super().__init__()
        self.n_layers = n_layers
        self.n_units = n_units
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation

        # Creating a list of linear layers with different numbers of units for each layer
        self.layers = nn.ModuleList([nn.Linear(3, n_units[0])])
        for i in range(1, n_layers):
            self.layers.append(nn.Linear(n_units[i - 1], n_units[i]))
        self.layers.append(nn.Linear(n_units[-1], 1))

    def forward(self, x):
        """Performs a forward pass on the input tensor.

        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, 3).

        Returns:
            torch.Tensor: The output tensor of shape (batch_size, 1).
        """
        # Looping over the hidden layers and applying the linear transformation and the activation function
        for layer in self.layers[:-1]:
            x = self.hidden_activation(layer(x))

        # Applying the linear transformation and the activation function on the output layer
        x = self.output_activation(self.layers[-1](x))

        # Returning the output tensor
        return x


## Setting the search space

In [17]:
# Defining a function to create a trial network and optimizer
def create_model(trial):
    """Creates a trial network and optimizer based on the sampled hyperparameters.

    Args:
        trial (optuna.trial.Trial): The trial object that contains the hyperparameters.

    Returns:
        tuple: A tuple of (net, loss_fn, optimizer, batch_size, n_epochs,
            scheduler), where net is the trial network,
            loss_fn is the loss function,
            optimizer is the optimizer,
            batch_size is the batch size,
            n_epochs is the number of epochs,
            scheduler is the learning rate scheduler.
    """

    # Sampling the hyperparameters from the search space
    n_layers = trial.suggest_int("n_layers", 1, 5)
    n_layers = trial.suggest_int("n_layers", 1, 5)
    n_units = [trial.suggest_int(f"n_units_{i}", 1, 256) for i in range(n_layers)]
    hidden_activation_name = trial.suggest_categorical("hidden_activation", ["ReLU", "Tanh", "Sigmoid"])
    output_activation_name = trial.suggest_categorical("output_activation", ["ReLU", "Tanh", "Sigmoid"])
    # Added another common loss function: CrossEntropyLoss
    loss_name = trial.suggest_categorical("loss", ["MSE", "MAE", "Huber", "CrossEntropy"])
    # Added another common optimizer: Adagrad
    optimizer_name = trial.suggest_categorical("optimizer", ["SGD", "Adam", "RMSprop", "Adagrad"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    batch_size = trial.suggest_int("batch_size", 1, 512)
    # Increased the maximum number of epochs that are tried
    n_epochs = trial.suggest_int("n_epochs", 10, 200)
    # Added the possibility of having different schedulers: CosineAnnealingLR and ReduceLROnPlateau
    scheduler_name = trial.suggest_categorical("scheduler", ["None", "StepLR", "ExponentialLR", "CosineAnnealingLR", "ReduceLROnPlateau"])

    # Creating the activation functions from their names
    if hidden_activation_name == "ReLU":
        hidden_activation = nn.ReLU()
    elif hidden_activation_name == "Tanh":
        hidden_activation = nn.Tanh()
    else:
        hidden_activation = nn.Sigmoid()

    if output_activation_name == "ReLU":
        output_activation = nn.ReLU()
    elif output_activation_name == "Tanh":
        output_activation = nn.Tanh()
    else:
        output_activation = nn.Sigmoid()

    # Creating the loss function from its name
    if loss_name == "MSE":
        loss_fn = nn.MSELoss()
    elif loss_name == "MAE":
        loss_fn = nn.L1Loss()
    elif loss_name == "Huber":
        loss_fn = nn.SmoothL1Loss()
    else:
        # Added creating the CrossEntropyLoss function
        loss_fn = nn.CrossEntropyLoss()

    # Creating the network with the sampled hyperparameters
    net = Net(n_layers, n_units, hidden_activation, output_activation).to(device) # Added moving the network to the device

    # Creating the optimizer from its name
    if optimizer_name == "SGD":
        optimizer = optim.SGD(net.parameters(), lr=lr)
    elif optimizer_name == "Adam":
        optimizer = optim.Adam(net.parameters(), lr=lr)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(net.parameters(), lr=lr)
    else:
        # Added creating the Adagrad optimizer
        optimizer = optim.Adagrad(net.parameters(), lr=lr)

    # Creating the learning rate scheduler from its name
    if scheduler_name == "StepLR":
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    elif scheduler_name == "ExponentialLR":
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    elif scheduler_name == "CosineAnnealingLR":
        # Added creating the CosineAnnealingLR scheduler
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    elif scheduler_name == "ReduceLROnPlateau":
        # Added creating the ReduceLROnPlateau scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=10)
    else:
        scheduler = None

    # Returning the network, the loss function, the optimizer, the batch size, the number of epochs and the scheduler
    return net, loss_fn, optimizer, batch_size, n_epochs, scheduler


## The train and eval loop

In [25]:
# Defining a function to train and evaluate a network on the train and test sets
def train_and_eval(net, loss_fn, optimizer, batch_size, n_epochs, scheduler):
    """Trains and evaluates a network on the train and test sets.

    Args:
        net (Net): The network to train and evaluate.
        loss_fn (torch.nn.Module): The loss function to use.
        optimizer (torch.optim.Optimizer): The optimizer to use.
        batch_size (int): The batch size to use.
        n_epochs (int): The number of epochs to train for.
        scheduler (torch.optim.lr_scheduler._LRScheduler or None): The learning rate scheduler to use.

    Returns:
        tuple: A tuple of (train_losses, test_losses,
            train_accuracies, test_accuracies), where train_losses and test_losses are lists of average losses per epoch for the train and test sets,
            train_accuracies and test_accuracies are lists of average accuracies per epoch for the train and test sets.
    """
    # Creating data loaders for train and test sets
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size)

    # Initializing lists to store the losses and accuracies for each epoch
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    # Looping over the epochs
    for epoch in range(n_epochs):
        # Setting the network to training mode
        net.train()
        # Initializing variables to store the total loss and correct predictions for the train set
        train_loss = 0.0
        train_correct = 0
        # Looping over the batches in the train set
        for x_batch, y_batch in train_loader:
            # Zeroing the gradients
            optimizer.zero_grad()
            # Forward pass
            y_pred = net(x_batch)
            # Reshaping the target tensor to match the input tensor
            y_batch = y_batch.view(-1, 1)
            # Computing the loss
            loss = loss_fn(y_pred, y_batch)
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            # Updating the total loss and correct predictions
            train_loss += loss.item() * x_batch.size(0)
            train_correct += torch.sum((y_pred - y_batch).abs() < 1e-3)

        # Setting the network to evaluation mode
        net.eval()
        # Initializing variables to store the total loss and correct predictions for the test set
        test_loss = 0.0
        test_correct = 0
        # Looping over the batches in the test set
        for x_batch, y_batch in test_loader:
            # Forward pass
            y_pred = net(x_batch)
            # Reshaping the target tensor to match the input tensor
            y_batch = y_batch.view(-1, 1)
            # Computing the loss
            loss = loss_fn(y_pred, y_batch)
            # Updating the total loss and correct predictions
            test_loss += loss.item() * x_batch.size(0)
            test_correct += torch.sum((y_pred - y_batch).abs() < 1e-3)

        # Computing the average losses and accuracies for the train and test sets
        train_loss /= len(x_train)
        test_loss /= len(x_test)
        train_accuracy = train_correct / len(x_train)
        test_accuracy = test_correct / len(x_test)

        # Appending the losses and accuracies to the lists
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)

        # Updating the learning rate scheduler with validation loss if applicable
        if scheduler is not None: # Added checking if scheduler is not None
            if isinstance(scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(test_loss) # Added passing validation loss as metric value
            else:
                scheduler.step()

        # Printing the epoch summary
        print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    # Returning the lists of losses and accuracies
    return train_losses, test_losses, train_accuracies, test_accuracies


In [20]:
# Defining a function to compute the objective value for a trial
def objective(trial):
    """Computes the objective value (final test loss) for a trial.

    Args:
        trial (optuna.trial.Trial): The trial object that contains the hyperparameters.

    Returns:
        float: The objective value (final test loss) for the trial.
    """
    # Creating a trial network and optimizer with the create_model function
    net, loss_fn, optimizer, batch_size, n_epochs, scheduler = create_model(trial)

    # Training and evaluating the network with the train_and_eval function
    train_losses, test_losses, train_accuracies, test_accuracies = train_and_eval(net, loss_fn, optimizer, batch_size, n_epochs, scheduler)

    # Returning the final test loss as the objective value
    return test_losses[-1]



## Finding the best hyperparameters with Optuna

In [24]:
# Creating a study object with Optuna
study = optuna.create_study(direction="minimize")

# Running the optimization for 100 trials
study.optimize(objective, n_trials=100)


[I 2023-05-04 19:10:46,328] A new study created in memory with name: no-name-bf198c00-97a8-4244-b55e-c2b08b70769c
/tmp/ipykernel_27400/1147161505.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)


Epoch 1: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 2: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 3: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 4: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 5: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 6: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 7: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 8: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 9: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 10: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 11: Train Loss: 0.0000, Train Accuracy: 0.0000, Test 

[I 2023-05-04 19:11:18,067] Trial 0 finished with value: 0.0 and parameters: {'n_layers': 1, 'n_units_0': 163, 'hidden_activation': 'ReLU', 'output_activation': 'ReLU', 'loss': 'CrossEntropy', 'optimizer': 'Adagrad', 'lr': 0.0011655728647735535, 'batch_size': 36, 'n_epochs': 138, 'scheduler': 'None'}. Best is trial 0 with value: 0.0.


Epoch 138: Train Loss: 0.0000, Train Accuracy: 0.0000, Test Loss: 0.0000, Test Accuracy: 0.0000
Epoch 1: Train Loss: 28.6625, Train Accuracy: 0.0000, Test Loss: 27.7361, Test Accuracy: 0.0005
Epoch 2: Train Loss: 28.5568, Train Accuracy: 0.0001, Test Loss: 27.7326, Test Accuracy: 0.0000
Epoch 3: Train Loss: 28.5458, Train Accuracy: 0.0000, Test Loss: 27.7309, Test Accuracy: 0.0000
Epoch 4: Train Loss: 28.5421, Train Accuracy: 0.0000, Test Loss: 27.7182, Test Accuracy: 0.0000
Epoch 5: Train Loss: 28.5373, Train Accuracy: 0.0000, Test Loss: 27.7138, Test Accuracy: 0.0000
Epoch 6: Train Loss: 28.5375, Train Accuracy: 0.0000, Test Loss: 27.7124, Test Accuracy: 0.0000
Epoch 7: Train Loss: 28.5354, Train Accuracy: 0.0000, Test Loss: 27.7244, Test Accuracy: 0.0000
Epoch 8: Train Loss: 28.5362, Train Accuracy: 0.0000, Test Loss: 27.7149, Test Accuracy: 0.0000
Epoch 9: Train Loss: 28.5348, Train Accuracy: 0.0000, Test Loss: 27.7137, Test Accuracy: 0.0000
Epoch 10: Train Loss: 28.5342, Train Acc

[I 2023-05-04 19:11:27,045] Trial 1 finished with value: 27.712963032722474 and parameters: {'n_layers': 3, 'n_units_0': 44, 'n_units_1': 77, 'n_units_2': 14, 'hidden_activation': 'ReLU', 'output_activation': 'Tanh', 'loss': 'MAE', 'optimizer': 'Adagrad', 'lr': 0.043225063648159846, 'batch_size': 313, 'n_epochs': 83, 'scheduler': 'CosineAnnealingLR'}. Best is trial 0 with value: 0.0.
[W 2023-05-04 19:11:27,128] Trial 2 failed with parameters: {'n_layers': 1, 'n_units_0': 188, 'hidden_activation': 'Tanh', 'output_activation': 'Sigmoid', 'loss': 'MAE', 'optimizer': 'Adagrad', 'lr': 0.0004411365914680884, 'batch_size': 251, 'n_epochs': 86, 'scheduler': 'ReduceLROnPlateau'} because of the following error: UnboundLocalError("local variable 'test_loss' referenced before assignment").
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykerne

Epoch 82: Train Loss: 28.5349, Train Accuracy: 0.0000, Test Loss: 27.7230, Test Accuracy: 0.0000
Epoch 83: Train Loss: 28.5357, Train Accuracy: 0.0000, Test Loss: 27.7130, Test Accuracy: 0.0005


UnboundLocalError: local variable 'test_loss' referenced before assignment

## Training with the best parameters

In [9]:

# Printing the best parameters and the best value
print("Best parameters:", study.best_params)
print("Best value:", study.best_value)

# Creating a network with the best parameters
best_net, best_loss_fn, best_optimizer, best_batch_size, best_n_epochs, best_scheduler = create_model(study.best_trial)

# Training and evaluating the network with the best parameters
best_train_losses, best_test_losses, best_train_accuracies, best_test_accuracies = train_and_eval(best_net, best_loss_fn, best_optimizer, best_batch_size, best_n_epochs, best_scheduler)

# Saving the network with the best parameters
torch.save(best_net.state_dict(), "best_net.pth")


ValueError: No trials are completed yet.

## Visualizing results

In [10]:

# Creating a tensorboard writer
writer = tbx.SummaryWriter()

# Logging the losses and accuracies for each epoch
for epoch in range(best_n_epochs):
    writer.add_scalars("Loss", {"Train": best_train_losses[epoch], "Test": best_test_losses[epoch]}, epoch + 1)
    writer.add_scalars("Accuracy", {"Train": best_train_accuracies[epoch], "Test": best_test_accuracies[epoch]}, epoch + 1)

# Closing the writer
writer.close()

# Plotting the predicted values and true values for a random sample of 1000 test points
indices = np.random.choice(len(x_test), size=1000)
x_sample = x_test[indices]
y_sample = y_test[indices]
y_pred = best_net(x_sample).detach().numpy()
plt.scatter(y_sample, y_pred, alpha=0.1)
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted Values vs True Values")
plt.savefig("pred_vs_true.png")
plt.figure()

# Plotting the bias (difference between predicted values and true values) for a random sample of 1000 test points
bias = y_pred - y_sample.numpy()
plt.hist(bias)
plt.xlabel("Bias")
plt.ylabel("Frequency")
plt.title("Bias Distribution")
plt.savefig("bias_dist.png")


NameError: name 'best_n_epochs' is not defined

## Saving data

### Pickle

In [11]:
import pickle

# Save the objects to a file
with open("best_objects.pkl", "wb") as f:
    pickle.dump(best_net, f)
    pickle.dump(best_loss_fn, f)
    pickle.dump(best_optimizer, f)
    pickle.dump(best_batch_size, f)
    pickle.dump(best_n_epochs, f)
    pickle.dump(best_scheduler, f)
    pickle.dump(best_train_losses, f)
    pickle.dump(best_test_losses, f)
    pickle.dump(best_train_accuracies, f)
    pickle.dump(best_test_accuracies, f)

NameError: name 'best_net' is not defined

### Pandas

Pandas is not suitable for saving complex objects like PyTorch models, loss functions, optimizers, or schedulers. Pandas is mainly used for saving tabular data, such as lists of numbers or strings. If you want to save these objects for later analysis, you may want to use pickle or torch.save instead. However, if you only want to save the hyperparameters and the metrics, you can use pandas.DataFrame to create a data frame with one row and multiple columns. You can then use pandas.to_csv to write the data frame to a CSV file. For example, you could use the following code:


In [13]:
import pandas as pd

# Get the names of the loss function, optimizer, and scheduler
loss_name = type(best_loss_fn).__name__
optimizer_name = type(best_optimizer).__name__
scheduler_name = type(best_scheduler).__name__ if best_scheduler else "None"

# Create a data frame with one row and multiple columns
data = pd.DataFrame({
    "hidden_activation": [best_net.hidden_activation.__class__.__name__],
    "output_activation": [best_net.output_activation.__class__.__name__],
    "n_units": [str(study.best_params["n_units"])],
    "loss": [loss_name],
    "optimizer": [optimizer_name],
    "lr": [best_optimizer.param_groups[0]["lr"]],
    "batch_size": [best_batch_size],
    "n_epochs": [best_n_epochs],
    "scheduler": [scheduler_name],
    "train_loss": [best_train_losses[-1]],
    "test_loss": [best_test_losses[-1]],
    "train_accuracy": [best_train_accuracies[-1]],
    "test_accuracy": [best_test_accuracies[-1]]
})

# Save the data frame to a CSV file
data.to_csv("best_data.csv", index=False)

NameError: name 'best_loss_fn' is not defined

### JSON to store study values 

In [17]:
import json

# Save the values to a file
with open("best_values.json", "w") as f:
    json.dump({"params": study.best_params, "value": study.best_value}, f)

# Load the values from the file
with open("best_values.json", "r") as f:
    best_values = json.load(f)

## Loading data

### Pickle

In [ ]:
import pickle

# Load the objects from the file
with open("best_objects.pkl", "rb") as f:
    best_net = pickle.load(f)
    best_loss_fn = pickle.load(f)
    best_optimizer = pickle.load(f)
    best_batch_size = pickle.load(f)
    best_n_epochs = pickle.load(f)
    best_scheduler = pickle.load(f)
    best_train_losses = pickle.load(f)
    best_test_losses = pickle.load(f)
    best_train_accuracies = pickle.load(f)
    best_test_accuracies = pickle.load(f)

### Loading the model

To load your saved best_net.pth, you need to create an instance of the same model class first, and then load the state dictionary using the torch.load and load_state_dict methods. For example, you could use the following code:

In [19]:
import json

# Load the values from the file
with open("best_values.json", "r") as f:
    best_values = json.load(f)

# Get the number of layers, the number of units, the hidden activation and the output activation
n_layers = best_values["params"]["n_layers"]
n_units = best_values["params"]["n_units"]
hidden_activation = best_values["params"]["hidden_activation"]
output_activation = best_values["params"]["output_activation"]

# Print the values
print("Number of layers:", n_layers)
print("Number of units:", n_units)
print("Hidden activation:", hidden_activation)
print("Output activation:", output_activation)

Number of layers: 1
Number of units: 6
Hidden activation: ReLU
Output activation: ReLU


In [ ]:
# Create an instance of the same model class
model = Net(n_layers, n_units, hidden_activation, output_activation)

# Load the state dictionary from the file
model.load_state_dict(torch.load("best_net.pth"))

# Set the model to evaluation mode
model.eval()
